In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import *

In [20]:
text = """들판에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [36]:
t = Tokenizer()
t.fit_on_texts([text])

In [39]:
vocabSize= len(t.word_index) +1  #  원핫인코딩이 필요

In [25]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        # print(sequence)
        # print('='*50)
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [30]:
sequences = pad_sequences(sequences, maxlen= maxLen, padding='pre') # 벡터의 길이를 맞춰주는 코드

In [32]:
x = sequences[:, :-1] # 마지막열은 제외한 거 
y = sequences[:, -1] # 마지막열

print(x, y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]] [ 3  1  4  5  1  7  1  9 10  1 11]


In [40]:
y = to_categorical(y, num_classes=vocabSize)
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [44]:
from keras.models import  Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [47]:
model = Sequential()
model.add(Embedding(vocabSize, 10, input_length=maxLen-1))
model.add(SimpleRNN(32))
model.add(Dense(vocabSize, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x, y, epochs = 200)

Epoch 1/200
1/1 [==============================] - 0s 997us/step - loss: 2.4965 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 997us/step - loss: 2.4860 - accuracy: 0.1818
Epoch 3/200
1/1 [==============================] - 0s 998us/step - loss: 2.4753 - accuracy: 0.1818
Epoch 4/200
1/1 [==============================] - 0s 957us/step - loss: 2.4644 - accuracy: 0.3636
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4532 - accuracy: 0.3636
Epoch 6/200
1/1 [==============================] - 0s 0s/step - loss: 2.4416 - accuracy: 0.3636
Epoch 7/200
1/1 [==============================] - 0s 998us/step - loss: 2.4295 - accuracy: 0.3636
Epoch 8/200
1/1 [==============================] - 0s 998us/step - loss: 2.4168 - accuracy: 0.3636
Epoch 9/200
1/1 [==============================] - 0s 2ms/step - loss: 2.4035 - accuracy: 0.3636
Epoch 10/200
1/1 [==============================] - 0s 998us/step - loss: 2.3895 - accuracy: 0.3636
Epoch 11/200

# '들판에' -> '들판에 있는 말이 뛰고 있다' 예측 문장 완성

In [57]:
def sentence_generation(model, t, cw, n):
    initWord = cw
    sentence = ''
    for _ in range(n): # 4번 반복
        encoded = t.texts_to_sequences([cw])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # print(encoded)
        result = model.predict_classes(encoded)
        print(result) # [3] '있는'의 인덱스
        for word, index in t.word_index.items(): # 워드랑 인덱스가 뽑힘
            if index == result:
                break
        cw = cw+' '+word
        sentence = sentence+' ' +word
    sentence = initWord + sentence
    return sentence

sentence_generation(model, t, '가는', 5)

[1]
[9]
[10]
[1]
[11]


'가는 말이 고와야 오는 말이 곱다'

In [58]:
import pandas as pd

In [62]:
df = pd.read_csv('딥러닝/nyt.csv')
df.head()['webURL']

0    https://www.nytimes.com/2018/04/24/sports/foot...
1    https://www.nytimes.com/2018/04/24/climate/epa...
2    https://www.nytimes.com/2018/04/24/dining/noma...
3    https://www.nytimes.com/2018/04/24/world/europ...
4    https://www.nytimes.com/2018/04/24/world/canad...
Name: webURL, dtype: object

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         1324 non-null   object
 1   articleWordCount  1324 non-null   int64 
 2   byline            1324 non-null   object
 3   documentType      1324 non-null   object
 4   headline          1324 non-null   object
 5   keywords          1324 non-null   object
 6   multimedia        1324 non-null   int64 
 7   newDesk           1324 non-null   object
 8   printPage         1324 non-null   int64 
 9   pubDate           1324 non-null   object
 10  sectionName       1324 non-null   object
 11  snippet           1324 non-null   object
 12  source            1324 non-null   object
 13  typeOfMaterial    1324 non-null   object
 14  webURL            1324 non-null   object
dtypes: int64(3), object(12)
memory usage: 155.3+ KB


In [66]:
df['headline'].isnull().sum()
df['headline'].isnull().values.any()


False

In [70]:
headline = []
headline.extend(list(df.headline.values))

In [71]:
headline = [n for n in headline if n != 'Unknown']
len(headline)

1214

In [73]:
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [74]:
def repreprocessing(s) :
    # s에 전달된 뉴스기사제목에 대해 전처리 (점 제거, 소문자)
    print(''.join(c for c in s if c not in punctuation).lower())
    #return result
text = [repreprocessing(x) for x in headline]
text[:5]

former nfl cheerleaders’ settlement offer 1 and a meeting with goodell
epa to unveil a new rule its effect less science in policymaking
the new noma explained
how a bag of texas dirt  became a times tradition
is school a place for selfexpression
commuter reprogramming
ford changed leaders looking for a lift it’s still looking
romney failed to win at utah convention but few believe he’s doomed
chain reaction
he forced the vatican to investigate sex abuse now he’s meeting with pope francis
in berlin artists find a home
the right stuff
jimmy carter knows what north korea wants
the truth is out there
new jersey ruling could reignite battle over churchstate separation
procrastinating
word  quiz dilatory
my lifethreatening bout with e coli food poisoning
choosing brexit a town yearned for its seafaring past and muddied its future
a quote disproved
hot stuff turns cold
at the top the pay gap may be gone
years of bizarre behavior but he got guns back
will the court stand up to mr trump
hope in

[None, None, None, None, None]